In [1]:
import pandas as pd
import numpy as np
import os
import math

In [15]:
# path
path = r"C:\Users\USYS671257\OneDrive - WSP O365\21_31000110.002_Chattanooga TPO Model\model outputs"

# inputs
countycorr = pd.read_csv(os.path.join(path, "county_districts_chattanooga.csv"))
tourdata  = pd.read_csv(os.path.join(path, "_tour_2.dat"), sep = '\t')
hhdata  = pd.read_csv(os.path.join(path, "_household_2.dat"), sep = '\t')
perdata = pd.read_csv(os.path.join(path, "_person_2.dat"), sep = '\t')

# parameters
excludeChildren5 = True

In [33]:
def prep_tourdata(tourdata, perdata):

    perdata = perdata[["hhno","pno","pptyp","psexpfac"]]
    tourdata = pd.merge(tourdata, perdata, on=["hhno","pno"], how="left")
    if excludeChildren5:
        tourdata = tourdata[tourdata["pptyp"]<8]

    tourdata["pdpurp2"] = np.where(tourdata.pdpurp>2, 3, tourdata.pdpurp)    
    tourdata["pdpurp2"] = np.where(tourdata.pdpurp==0, 0, tourdata.pdpurp2)       
    tourdata["pdpurp2"] = np.where(tourdata.parent>0, 4, tourdata.pdpurp2)

    tourdata["deppdhr"] = (tourdata["tlvdest"]/100).apply(math.trunc)
    tourdata["deppdmin"] = tourdata["tlvdest"]-tourdata["deppdhr"]*100

    if len(tourdata[tourdata["deppdmin"]>60]["deppdmin"])>0:
        tourdata["deptime"] = tourdata["tlvdest"]/60
        tourdata["arrtime"] = tourdata["tardest"]/60
    else:
        tourdata["deptime"] = tourdata["deppdhr"]+tourdata["deppdmin"]/60
        tourdata["arrpdhr"] = (tourdata["tardest"]/100).apply(math.trunc)
        tourdata["arrpdmin"] = tourdata["tardest"]-tourdata["arrpdhr"]*100
        tourdata["arrtime"] = tourdata["arrpdhr"]+tourdata["arrpdmin"]/60

    tourdata["durdest"] = tourdata["deptime"]-tourdata["arrtime"]

    cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    tourdata["arrtimecat"] = pd.cut(tourdata["arrtime"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])
    tourdata["deptimecat"] = pd.cut(tourdata["deptime"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])
    cats = [0] + np.arange(0.5, 25, 0.5).tolist()
    tourdata["durdestcat"] = pd.cut(tourdata["durdest"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])

    return tourdata
    
    

def summary_tour_tod(dstourdata, sum_by_var):
    
    """  
        Work 1 
        School 2 
        Other 3
        Workbased 4
    """

    if sum_by_var == "durdestcat":
        cats = [0] + np.arange(0.5, 25, 0.5).tolist()
    else:
        cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    
    index_order = cats[:-1]
    summary = (dstourdata.groupby([sum_by_var,"pdpurp2"])["psexpfac"].
                          sum().
                          reset_index().
                          pivot_table(values='psexpfac', 
                                     index=sum_by_var,
                                     columns="pdpurp2",
                                     fill_value=0).
                         reindex(index=index_order).
                         fillna(0))
    return summary



def summary_tour_tod_purpose(dstourdata, sum_by_var, purpose):
    
    """  
        Work 1 
        School 2 
        Other 3
        Workbased 4
        1 Full time worker, 
        2 Part time worker, 
        3 Non working adult age 65+, 
        4 Non working adult age<65, 
        5 University student, 
        6 High school student age 16+, 
        7 Child age 5-15, 
        8 Child age 0-4
    """
    dstourdata = dstourdata[dstourdata["pdpurp2"]==purpose]
    cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    index_order = cats[:-1]
    column_order = list(range(1, 9))
    summary = (dstourdata.groupby([sum_by_var,"pptyp"])["psexpfac"].
                          sum().
                          reset_index().
                          pivot_table(values='psexpfac', 
                                     index=sum_by_var,
                                     columns="pptyp",
                                     fill_value=0).
                         reindex(index=index_order,
                                 columns=column_order).
                         fillna(0))
    return summary

In [17]:
# clean up tour data
dstourdata = prep_tourdata(tourdata, perdata)

In [20]:
# Tour_Arrival_Time
summary_tour_tod(dstourdata, "arrtimecat")

pdpurp2,1,2,3,4
arrtimecat,,,,
0.0,232.0,0.0,927.0,6.0
3.0,592.0,0.0,387.0,1.0
3.5,1015.0,1.0,386.0,11.0
4.0,1322.0,0.0,317.0,27.0
4.5,1506.0,0.0,307.0,39.0
5.0,1561.0,0.0,218.0,85.0
5.5,1831.0,19.0,294.0,111.0
6.0,10267.0,968.0,1971.0,2.0
6.5,10944.0,1117.0,1844.0,15.0


In [21]:
# Tour_Departure_Time
summary_tour_tod(dstourdata, "deptimecat")

pdpurp2,1,2,3,4
deptimecat,,,,
0.0,2199.0,3.0,2809.0,6.0
3.0,5.0,0.0,120.0,1.0
3.5,5.0,0.0,263.0,4.0
4.0,18.0,0.0,264.0,9.0
4.5,26.0,0.0,260.0,19.0
5.0,51.0,0.0,269.0,42.0
5.5,58.0,0.0,230.0,65.0
6.0,108.0,0.0,895.0,63.0
6.5,195.0,2.0,1149.0,62.0


In [24]:
# Tour_Duration
summary_tour_tod(dstourdata, "durdestcat")

pdpurp2,1,2,3,4
durdestcat,,,,
0.0,3959.0,2411.0,111946.0,10443.0
0.5,2685.0,1309.0,49472.0,4487.0
1.0,2820.0,1182.0,41161.0,3585.0
1.5,3032.0,1254.0,27718.0,2136.0
2.0,3336.0,1353.0,17538.0,1139.0
2.5,3997.0,1997.0,11635.0,963.0
3.0,4791.0,2593.0,7041.0,852.0
3.5,5348.0,2616.0,5048.0,675.0
4.0,5659.0,2912.0,3596.0,524.0


In [37]:
# Arrival_Time_Work
summary_tour_tod_purpose(dstourdata, "arrtimecat", 1)

pptyp,1,2,3,4,5,6,7,8
arrtimecat,,,,,,,,
0.0,165.0,45.0,0.0,0.0,6.0,16.0,0.0,0.0
3.0,576.0,13.0,0.0,0.0,3.0,0.0,0.0,0.0
3.5,981.0,30.0,0.0,0.0,2.0,2.0,0.0,0.0
4.0,1248.0,71.0,0.0,0.0,2.0,1.0,0.0,0.0
4.5,1409.0,92.0,0.0,0.0,5.0,0.0,0.0,0.0
5.0,1457.0,99.0,0.0,0.0,5.0,0.0,0.0,0.0
5.5,1698.0,123.0,0.0,0.0,7.0,3.0,0.0,0.0
6.0,9486.0,722.0,0.0,0.0,47.0,12.0,0.0,0.0
6.5,10038.0,822.0,0.0,0.0,70.0,14.0,0.0,0.0


In [35]:
# Depart_Time_Work
summary_tour_tod_purpose(dstourdata, "deptimecat", 1)

pptyp,1,2,3,4,5,6,7,8
deptimecat,,,,,,,,
0.0,1841.0,246.0,0.0,0.0,56.0,56.0,0.0,0.0
3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3.5,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4.5,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,46.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
5.5,42.0,15.0,0.0,0.0,1.0,0.0,0.0,0.0
6.0,80.0,27.0,0.0,0.0,1.0,0.0,0.0,0.0
6.5,143.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# Arrival_Time_School
summary_tour_tod_purpose(dstourdata, "arrtimecat", 2)

pptyp,1,2,3,4,5,6,7,8
arrtimecat,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.5,0.0,0.0,0.0,0.0,0.0,5.0,14.0,0.0
6.0,1.0,0.0,0.0,0.0,38.0,225.0,704.0,0.0
6.5,2.0,0.0,0.0,0.0,45.0,245.0,825.0,0.0


In [39]:
# Depart_Time_School
summary_tour_tod_purpose(dstourdata, "deptimecat", 2)

pptyp,1,2,3,4,5,6,7,8
deptimecat,,,,,,,,
0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
